In [1]:
import os
import subprocess
import pandas as pd #pip install pandas
import xlwt #pip install xlwt
from openpyxl import load_workbook #pip install openpyxldd

In [ ]:
IOD = {"Серийный номер принтера": "1.3.6.1.2.1.43.5.1.1.17.1",
            "Цвет тонера": "1.3.6.1.2.1.43.12.1.1.4.1.1",
            "Максимальное число копий тонера": "1.3.6.1.2.1.43.11.1.1.8.1.1",
            "Оставшееся число копий тонера": "1.3.6.1.2.1.43.11.1.1.9.1.1",
            "Кол-во напечатанных страниц": "1.3.6.1.2.1.43.10.2.1.4.1.1"
            }

In [14]:
snmp_dir = "C:/Users/Никита Широкопетлев/Documents/SnmpGet/"
def get_IOD(wb, ser):
    IOD = {}
    work_sheet = wb['IOD']
    
    return IOD


def get_info_row(wb, row, cell_ser = 3, cell_IP = 2):
    result = [] 
    IOD = get_IOD(wb, ser=row[cell_ser])
    for key, value in IOD.items():
        res = subprocess.check_output(snmp_dir + "SnmpGet.exe -r:" + str(row[cell_IP]) + 
                                      ' -o:' + value).decode("cp866").split('\n')
        result.append(res[-2].split('=')[-1])
    return result

def search_cell(name_cell, work_sheet):
    for cell_ser in range(1, 100):
        if work_sheet.cell(row = 1, column = cell_ser).value == name_cell:
            break
    print('Столбец "', name_cell, '": ', cell_ser, sep = '')
    return cell_ser

In [15]:
wb = load_workbook('./printers.xlsx')
print(wb.sheetnames)
work_sheet = wb['MAIN']
cell_ser = search_cell('Серийный номер', work_sheet)
cell_IP = search_cell('IP', work_sheet)

['MAIN', 'Sheet1', 'AL75023437', 'CNB1L7J286', 'VCF7538360', 'E75373J7N561836', 'VCG8Y01076', 'VCG9118770', 'VCG8887941', 'ZDP1BJFF9000RLT', 'Лист10']
Столбец "Серийный номер": 2
Столбец "IP": 3


In [41]:
start_row = 2
cur_row = start_row
for row in work_sheet.iter_rows(min_row=2, values_only=True):
    try:
        result = get_info_row(wb, row, cell_ser - 1 , cell_IP - 1)
        work_sheet.cell(row=cur_row, column=4).value = int(result[3].strip())/int(result[2].strip())
    except:
        pass
    cur_row += 1

['MAIN', 'Sheet1', 'AL75023437', 'CNB1L7J286', 'VCF7538360', 'E75373J7N561836', 'VCG8Y01076', 'VCG9118770', 'VCG8887941', 'ZDP1BJFF9000RLT', 'Лист10']
Столбец серийных номеров: 2
Столбец IP адресов: 3


In [43]:
wb.save('./printers.xlsx')